# Estimate current camera coverage and how many we want for better coverage

In [ ]:
import os
from glob import glob
import pandas as pd
import rioxarray as rxr
from shapely.geometry import shape, Polygon
from shapely.geometry.collection import GeometryCollection
from shapely import get_coordinates
from shapely import affinity, union_all
import matplotlib.pyplot as plt
import geopandas as gpd
import xarray as xr
import numpy as np
import rasterio as rio
from tqdm import tqdm

inputs_folder = '/Users/rdcrlrka/Research/Soo_locks/inputs'
cam_positions_file = os.path.join(inputs_folder, 'cams.txt')
ortho_files = sorted(glob(os.path.join(inputs_folder, '..', 'outputs', 'soo_locks_photogrammetry_20251001171000', 'orthoimages', '*.tiff')))
refdem_file = os.path.join(inputs_folder, 'lidar_DSM_filled_cropped.tif')

output_folder = os.path.join(inputs_folder, '..', 'add_cameras')
os.makedirs(output_folder, exist_ok=True)

# Load camera positions
cams = pd.read_csv(cam_positions_file, sep=' ', header=0)
cams['channel'] = [x.split('_')[1] for x in cams['img_name']]


## Helper functions

In [ ]:
def calculate_image_footprint(raster_file):
    # create a mask of data coverage
    raster = rxr.open_rasterio(raster_file).isel(band=0)
    crs, transform = raster.rio.crs, raster.rio.transform()
    if 'DSM' in raster_file:
        raster = xr.where(raster!=-9999, raster, np.nan)
    else:
        raster = xr.where(raster > 0, raster, np.nan)
    mask = raster.notnull()

    # vectorize the mask
    shape_gen = (
        (shape(s), v) 
        for s, v in 
        rio.features.shapes(mask.values.astype(np.int8), transform=transform)
        ) 
    gdf = gpd.GeoDataFrame(dict(zip(["geometry", "mask"], zip(*shape_gen))), crs=crs)

    # use just the exterior of all polygons
    exteriors = [x.exterior for x in gdf['geometry']]
    exterior_polys = [Polygon(get_coordinates(x)) for x in exteriors]
    gdf['geometry'] = exterior_polys

    # add the camera "channel"
    gdf['channel'] = os.path.basename(raster_file).split('_')[0]
    
    # use the largest mask polygon
    gdf['area'] = [x.area for x in gdf.geometry]
    gdf = gdf.sort_values(by='area', ascending=False).reset_index(drop=True)
    gdf = gdf.iloc[0:3]
    gdf = gdf.loc[gdf['mask']==1].reset_index(drop=True)   

    # make sure there's only one remaining
    gdf = gpd.GeoDataFrame(gdf.iloc[0]).transpose()

    # buffer the polygon slightly to remove sharp divots
    gdf['geometry'] = [x.buffer(0.1, join_style=1) for x in gdf['geometry']]

    # recalculate area
    gdf['area'] = [x.area for x in gdf['geometry']]

    gdf = gdf.set_geometry('geometry', crs=crs)

    return gdf


def calculate_image_overlap(bounds_gdf):
    overlap_list = []
    for i in tqdm(range(len(bounds_gdf))):
        for j in range(len(bounds_gdf)):
            if i==j:
                continue
            poly1 = bounds_gdf.loc[i, 'geometry']
            poly2 = bounds_gdf.loc[j, 'geometry']
            intersection = poly1.intersection(poly2)  
            if intersection.is_empty:
                continue
            df = pd.DataFrame({
                'channel_a': bounds_gdf.loc[i, 'channel'],
                'channel_b': bounds_gdf.loc[j, 'channel'],
                'overlap_area': intersection.area,
                'overlap_geometry': intersection
            }, index=[i])
            overlap_list += [df]
    overlap_df = pd.concat(overlap_list)
    overlap_gdf = gpd.GeoDataFrame(overlap_df, geometry=overlap_df['overlap_geometry'], crs=bounds_gdf.crs)
    return overlap_gdf


def get_coords_between_cams(ch1, ch2):
    cam1 = cams.loc[cams['channel']==ch1]
    cam2 = cams.loc[cams['channel']==ch2]
    xcoord = np.nanmean([cam1['X'].values[0], cam2['X'].values[0]])
    ycoord = np.nanmean([cam1['Y'].values[0], cam2['Y'].values[0]])
    return (xcoord, ycoord)


def calculate_no_coverage(model_space, bounds):
    no_coverage = model_space['geometry'][0]
    for _,row in bounds.iterrows():
        no_coverage = no_coverage.difference(row['geometry'])
    return no_coverage


def create_new_footprint(ref_poly, ref_cam, new_cam, model_gdf):
    # calculate the coordinate shift using cam_positions
    xshift = new_cam['X'] - ref_cam['X']
    yshift = new_cam['Y'] - ref_cam['Y'] 
    # shift the polygon coordinates
    new_poly = affinity.translate(ref_poly, xoff=xshift, yoff=yshift)
    # clip to model space
    new_poly = new_poly.intersection(model_gdf['geometry'][0])

    return new_poly


def calculate_specs_from_new_coords(new_coords, cams, bounds, model_space):
    cams_new = pd.DataFrame({
        'img_name': ['None']*len(new_coords),
        'X': new_coords[:,0],
        'Y': new_coords[:,1],
        'X_std': [0.1]*len(new_coords),
        'Y_std': [0.1]*len(new_coords),
        'Z_std': [0.1]*len(new_coords),
        'channel': ['NEW']*len(new_coords)
    })
    cams_new_full = pd.concat([cams, cams_new]).reset_index(drop=True)

    # define a reference camera and image footprint polygon
    ref_poly = bounds['geometry'].iloc[-1]
    ref_cam = cams[['X', 'Y']].iloc[-1]

    # estimate new image footprints
    bounds_new_list = []
    for i in range(len(cams_new)):
        new_cam = cams_new[['X', 'Y']].iloc[i]
        new_poly = create_new_footprint(ref_poly, ref_cam, new_cam, model_space)
        bounds_new_list += [gpd.GeoDataFrame({'geometry': [new_poly]})]

    bounds_new_df = pd.concat(bounds_new_list)
    bounds_new_gdf = gpd.GeoDataFrame(bounds_new_df, geometry=bounds_new_df['geometry'], crs="EPSG:32619").reset_index(drop=True)
    bounds_new_full_gdf = pd.concat([bounds, bounds_new_gdf]).reset_index(drop=True)

    # recalculate overlap
    overlap_new_gdf = calculate_image_overlap(bounds_new_full_gdf)
    # identify model space with no coverage
    no_coverage_new_full = calculate_no_coverage(model_space, bounds_new_full_gdf)
    print('No coverage area = ', np.round(no_coverage_new_full.area,1), 'm^2')

    return cams_new, cams_new_full, bounds_new_gdf, bounds_new_full_gdf, overlap_new_gdf, no_coverage_new_full

    
def plot_model_coverage(axis, model_space, bounds, overlap, cam_positions, no_coverage,
                        bounds_color='#7570b3', overlap_color='#7570b3', missing_color='#d95f02', new_cam_color='#e7298a'):
    # model space
    model_space.plot(ax=axis, edgecolor='k', facecolor='None', linewidth=2)
    # image footprints
    bounds.plot(ax=axis, edgecolor=bounds_color, linewidth=2, linestyle='--', facecolor='None', legend=False)
    # image overlap
    overlap.plot(ax=axis, color=overlap_color, alpha=0.5, legend=False)
    # missing model coverage
    no_coverage_gdf = gpd.GeoDataFrame(geometry=[no_coverage], crs="EPSG:32619")
    no_coverage_gdf.plot(ax=axis, edgecolor='None', facecolor=missing_color, alpha=0.5, legend=False)
    # current and NEW camera positions
    cam_positions_current = cam_positions.loc[cam_positions['channel']!='NEW']
    axis.plot(cam_positions_current['X'].values, cam_positions_current['Y'].values, '*k', markersize=8, label='Cameras')
    cam_positions_new = cam_positions.loc[cam_positions['channel']=='NEW']
    axis.plot(cam_positions_new['X'].values, cam_positions_new['Y'].values, '*', 
              markerfacecolor=new_cam_color, markeredgecolor='w', linewidth=0.5, markersize=12, label='NEW cameras')
    # dummy points for legend
    xmin, xmax = axis.get_xlim()
    ymin, ymax = axis.get_ylim()
    axis.plot(1e3, 1e3, '-k', linewidth=2,label='Model space')
    axis.plot(1e3, 1e3, '--', linewidth=2, color=bounds_color, label='Image footprint')
    axis.plot(1e3, 1e3, 's', markerfacecolor=overlap_color, alpha=0.5, markeredgecolor='None', markersize=10, label='Image overlap')
    axis.plot(1e3, 1e3, 's', markerfacecolor=missing_color, alpha=0.5, markeredgecolor='None', markersize=10, label='No coverage')
    axis.set_xlim(xmin, xmax)
    axis.set_ylim(ymin, ymax)
    return

def save_specs_los(bounds_new, cams_new, out_file, iref):
    # Estimate FOV
    rotation = -13
    yaw = 360 + rotation
    cam_height = float(cams['Z'].mean()) + 8
    fov_v = 110
    fov_h = 120
    # poly_rotated = affinity.rotate(bounds_new['geometry'][iref], rotation)
    # # vertical FOV
    # h = max(poly_rotated.exterior.coords.xy[1]) - min(poly_rotated.exterior.coords.xy[1])
    # fov_v = np.abs(np.rad2deg(np.tan((h/2)/cam_height)) * 2)
    # # horizontal FOV
    # w = max(poly_rotated.exterior.coords.xy[0]) - min(poly_rotated.exterior.coords.xy[0])
    # fov_h = np.abs(np.rad2deg(np.tan((w/2)/cam_height)) * 2)

    specs_list = []
    for i,row in bounds_new.iterrows():
        cam = cams_new.iloc[i]
        # compile in dataframe
        df = pd.DataFrame({
            'new_cam_number': i+1,
            'X': cam['X'],
            'Y': cam['Y'],
            'Z': cam_height,
            'FOV_vertical': fov_v,
            'FOV_horizontal': fov_h,
            'roll': 0,
            'pitch': 0,
            'yaw': yaw
        }, index=[i])
        specs_list += [df]

    specs_df = pd.concat(specs_list)
    specs_df = specs_df.round(2)

    # save
    specs_df.to_csv(out_file, header=True, index=False)
    print("New camera specs saved to file:", out_file)



## Calculate current image overlap

In [ ]:
print('Creating polygon of model space')
model_space_gdf = calculate_image_footprint(refdem_file)

print('Calculating image footprints')
gdf_list = []
for ortho_file in tqdm(ortho_files):
    gdf = calculate_image_footprint(ortho_file)
    gdf_list += [gdf]
bounds_df = pd.concat(gdf_list).reset_index(drop=True)
bounds_gdf = gpd.GeoDataFrame(bounds_df, geometry=bounds_df['geometry'], crs=gdf.crs)

print('Calculating overlap areas')
overlap_gdf = calculate_image_overlap(bounds_gdf)

print('Calculating area with no coverage')
no_coverage = calculate_no_coverage(model_space_gdf, bounds_gdf)

## Add 8 more cameras

In [ ]:
new_coords = np.array([
    (-22, 59),      # north 1
    (-17, 61),      # north 2
    get_coords_between_cams('ch04', 'ch05'),
    # get_coords_between_cams('ch06', 'ch07'), # BLOCKED BY WALL
    get_coords_between_cams('ch08', 'ch09'),
    get_coords_between_cams('ch10', 'ch11'),
    get_coords_between_cams('ch13', 'ch14'),
    get_coords_between_cams('ch14', 'ch15'),
    get_coords_between_cams('ch15', 'ch16')
])

(cams_new, cams_new_full, bounds_new_gdf, 
 bounds_new_full_gdf, overlap_new_gdf, 
 no_coverage_new_full) = calculate_specs_from_new_coords(new_coords, cams, bounds_gdf, model_space_gdf)

# Plot results
fig, ax = plt.subplots(1, 2, figsize=(10,12))
plot_model_coverage(ax[0], model_space_gdf, bounds_gdf, overlap_gdf, cams, no_coverage)
ax[0].set_title('Current camera coverage')
plot_model_coverage(ax[1], model_space_gdf, bounds_new_full_gdf, overlap_new_gdf, cams_new_full, no_coverage_new_full)
# plot the wall location
xcoord, ycoord = get_coords_between_cams('ch06', 'ch07')
ax[1].plot(xcoord, ycoord, 'xr', markersize=8, linewidth=2, label='Wall')
ax[1].set_title('Add 8 new cameras')
ax[1].legend(loc='lower left')
plt.show()

# Save figure
fig_file = os.path.join(output_folder, 'add_8_cameras.png')
fig.savefig(fig_file, dpi=300, bbox_inches='tight')
print('Figure saved to file:', fig_file)

# Save new specs
out_file = os.path.join(output_folder, 'add_8_cams_specs.csv')
save_specs_los(bounds_new_gdf, cams_new, out_file, iref=5)

## Add 11 more cameras

In [ ]:
new_coords = np.array([
    (-22, 59),      # north 1
    (-17, 61),      # north 2
    get_coords_between_cams('ch04', 'ch05'),
    get_coords_between_cams('ch05', 'ch06'),
    # get_coords_between_cams('ch06', 'ch07'), # BLOCKED BY WALL
    get_coords_between_cams('ch08', 'ch09'),
    get_coords_between_cams('ch10', 'ch11'),
    get_coords_between_cams('ch11', 'ch12'),
    get_coords_between_cams('ch12', 'ch13'),
    get_coords_between_cams('ch13', 'ch14'),
    get_coords_between_cams('ch14', 'ch15'),
    get_coords_between_cams('ch15', 'ch16')
])

(cams_new, cams_new_full, bounds_new_gdf, 
 bounds_new_full_gdf, overlap_new_gdf, 
 no_coverage_new_full) = calculate_specs_from_new_coords(new_coords, cams, bounds_gdf, model_space_gdf)

# Plot results
fig, ax = plt.subplots(1, 2, figsize=(10,12))
plot_model_coverage(ax[0], model_space_gdf, bounds_gdf, overlap_gdf, cams, no_coverage)
ax[0].set_title('Current camera coverage')
plot_model_coverage(ax[1], model_space_gdf, bounds_new_full_gdf, overlap_new_gdf, cams_new_full, no_coverage_new_full)
# plot the wall location
xcoord, ycoord = get_coords_between_cams('ch06', 'ch07')
ax[1].plot(xcoord, ycoord, 'xr', markersize=8, linewidth=2, label='Wall')
ax[1].set_title('Add 11 new cameras')
ax[1].legend(loc='lower left')
plt.show()

# Save figure
fig_file = os.path.join(output_folder, 'add_11_cameras.png')
fig.savefig(fig_file, dpi=300, bbox_inches='tight')
print('Figure saved to file:', fig_file)

# Save new specs
out_file = os.path.join(output_folder, 'add_11_cams_specs.csv')
save_specs_los(bounds_new_gdf, cams_new, out_file, iref=5)

## Add 15 more cameras

In [ ]:
# place 16 more cameras
new_coords = np.array([
    (-22, 59),      # north 1
    (-17, 61),      # north 2
    get_coords_between_cams('ch02', 'ch03'),
    get_coords_between_cams('ch03', 'ch04'),
    get_coords_between_cams('ch04', 'ch05'),
    get_coords_between_cams('ch05', 'ch06'),
    # get_coords_between_cams('ch06', 'ch07'), # BLOCKED BY WALL
    get_coords_between_cams('ch07', 'ch08'),
    get_coords_between_cams('ch08', 'ch09'),
    get_coords_between_cams('ch09', 'ch10'),
    get_coords_between_cams('ch10', 'ch11'),
    get_coords_between_cams('ch11', 'ch12'),
    get_coords_between_cams('ch12', 'ch13'),
    get_coords_between_cams('ch13', 'ch14'),
    get_coords_between_cams('ch14', 'ch15'),
    get_coords_between_cams('ch15', 'ch16')
])

(cams_new, cams_new_full, bounds_new_gdf, 
 bounds_new_full_gdf, overlap_new_gdf, 
 no_coverage_new_full) = calculate_specs_from_new_coords(new_coords, cams, bounds_gdf, model_space_gdf)

# Plot results
fig, ax = plt.subplots(1, 2, figsize=(10,12))
plot_model_coverage(ax[0], model_space_gdf, bounds_gdf, overlap_gdf, cams, no_coverage)
ax[0].set_title('Current camera coverage')
plot_model_coverage(ax[1], model_space_gdf, bounds_new_full_gdf, overlap_new_gdf, cams_new_full, no_coverage_new_full)
# plot the wall location
xcoord, ycoord = get_coords_between_cams('ch06', 'ch07')
ax[1].plot(xcoord, ycoord, 'xr', markersize=8, linewidth=2, label='Wall')
ax[1].set_title('Add 15 new cameras')
ax[1].legend(loc='lower left')
plt.show()

# Save figure
fig_file = os.path.join(output_folder, 'add_15_cameras.png')
fig.savefig(fig_file, dpi=300, bbox_inches='tight')
print('Figure saved to file:', fig_file)

# Save new specs
out_file = os.path.join(output_folder, 'add_15_cams_specs.csv')
save_specs_los(bounds_new_gdf, cams_new, out_file, iref=5)